<a href="https://colab.research.google.com/github/Br0d0brey/GoogleColabFiles/blob/main/nlp_imdb_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline
import re
import math
import string
from collections import Counter
import requests


Populating the interactive namespace from numpy and matplotlib


In [ ]:
TEXT = requests.get('https://norvig.com/big.txt').text
len(TEXT)

6488666

In [ ]:
def tokens(text):
    """Возвращает список токенов (подряд идущих буквенных последовательностей) в тексте. 
       Текст при этом приводится к нижнему регистру."""
    return re.findall(r'[a-z]+', text.lower())

In [ ]:
tokens('This is: A test, 1, 2, 3, this is.')

['this', 'is', 'a', 'test', 'this', 'is']

In [ ]:
WORDS = tokens(TEXT)
len(WORDS)



1105285

In [ ]:
print(WORDS[:10])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


In [ ]:
def sample(bag, n=10):
    "Sample a random n-word sentence from the model described by the bag of words."
    return ' '.join(random.choice(bag) for _ in range(n))

In [ ]:
sample(WORDS)



'the and degenerative to only standards domestic of for s'

In [ ]:
Counter(tokens('Is this a test? It is a test!'))

Counter({'a': 2, 'is': 2, 'it': 1, 'test': 2, 'this': 1})

In [ ]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [ ]:
def correct(word):
    "Поиск лучшего исправления ошибки для данного слова."
    # предрассчитать edit_distance==0, затем 1, затем 2; в противном случае оставить слово "как есть".
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [ ]:
def known(words):
    "Вернуть подмножество слов, которое есть в нашем словаре."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Вернуть все строки, которые находятся на edit_distance == 0 от word (т.е., просто само слово)."
    return {word}

def edits2(word):
    "Вернуть все строки, которые находятся на edit_distance == 2 от word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [ ]:
def edits1(word):
    "Возвращает список всех строк на расстоянии edit_distance == 1 от word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Возвращает список всех возможных разбиений слова на пару (a, b)."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'



In [ ]:
splits('wird')

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]

In [ ]:
print(edits0('wird'))

{'wird'}


In [ ]:
print(edits1('wird'))



{'wircd', 'wirdb', 'oird', 'wikd', 'wqrd', 'wimd', 'wirv', 'wqird', 'wiard', 'gwird', 'wifrd', 'wirod', 'qwird', 'woird', 'wid', 'whrd', 'wiro', 'wizrd', 'hird', 'wirdf', 'aird', 'iwrd', 'iird', 'iwird', 'wijrd', 'wirm', 'wirdk', 'wvrd', 'wcrd', 'wuird', 'awird', 'wimrd', 'vwird', 'wira', 'winrd', 'wixrd', 'wbird', 'mird', 'wijd', 'cird', 'swird', 'wirqd', 'bwird', 'wwrd', 'wipd', 'wirgd', 'xwird', 'wierd', 'widrd', 'wiyd', 'kwird', 'wibrd', 'wpird', 'wprd', 'wyird', 'wirb', 'werd', 'uwird', 'wikrd', 'wirn', 'wnrd', 'widr', 'uird', 'wirda', 'wirnd', 'owird', 'wkrd', 'wirw', 'wiqrd', 'wirkd', 'wmrd', 'wirdv', 'sird', 'xird', 'wirds', 'wirz', 'wirdn', 'wirpd', 'wurd', 'wxrd', 'wirjd', 'wiqd', 'wzrd', 'wkird', 'wirg', 'wirld', 'wirdz', 'wivrd', 'wiyrd', 'wiud', 'word', 'wtird', 'wirdl', 'wirde', 'wilrd', 'dird', 'wyrd', 'wirbd', 'wirwd', 'waird', 'bird', 'wicd', 'wifd', 'wiwd', 'wbrd', 'wjrd', 'vird', 'wirdd', 'wirtd', 'wmird', 'wirdi', 'wfrd', 'cwird', 'wiord', 'wirhd', 'whird', 'ird', '

In [ ]:
print(len(edits2('wird')))

24254


In [ ]:
tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')



['speling',
 'errurs',
 'in',
 'somethink',
 'whutever',
 'unusuel',
 'misteakes',
 'everyware']

In [ ]:
list(map(correct, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')))

['spelling',
 'errors',
 'in',
 'something',
 'whatever',
 'unusual',
 'mistakes',
 'everywhere']

In [ ]:
list(map(correct, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')))

['spelling',
 'errors',
 'in',
 'something',
 'whatever',
 'unusual',
 'mistakes',
 'everywhere']

In [ ]:
def correct_text(text):
    "Исправить все слова с опечатками в тексте."
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    "Исправить слово word в match-группе, сохранив регистр: upper/lower/title."
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    "Возвращает функцию регистра по тексту: upper, lower, title, или str."
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [ ]:
list(map(case_of, ['UPPER', 'lower', 'Title', 'CamelCase']))

[<method 'upper' of 'str' objects>,
 <method 'lower' of 'str' objects>,
 <method 'title' of 'str' objects>,
 str]

In [ ]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?')



'Spelling Errors IN something. Whatever; unusual mistakes?'

In [ ]:
correct_text('Intense Tropical Cyclone Batsirai was a deadly cyclone in 2022 that became the strongest tropical cyclone to strike Madagascar since Cyclone Enawo in 2017 and made landfall shortly after Tropical Storm Ana, which heavily impacted Madagascar earlier in the 2021–22 South-West Indian Ocean cyclone season.[1][2] The second tropical disturbance, the first tropical cyclone, and the first intense tropical cyclone of the season, Batsirai originated from a tropical disturbance that was first noted on 24 January. It fluctuated in intensity and became a moderate tropical storm on 27 January, after which it unexpectedly rapidly intensified into an intense tropical cyclone. It then weakened and struggled to intensify through the coming days due to present wind shear and dry air, where it weakened after some time. Afterward, it entered much more favorable conditions, rapidly intensified yet again to a high-end Category 4 cyclone on the Saffir–Simpson scale while moving towards Madagascar. The storm underwent an eyewall replacement cycle the next day, and fluctuated in intensity before making landfall in Madagascar as a Category 3 cyclone, later rapidly weakening due to the mountainous terrain on the island.')

'Intense Tropical Cycle Batsirai was a deadly cycle in 2022 that became the strongest tropical cycle to strike Madagascar since Cycle Enact in 2017 and made windfall shortly after Tropical Storm Ana, which heavily impacted Madagascar earlier in the 2021–22 South-West Indian Ocean cycle season.[1][2] The second tropical disturbance, the first tropical cycle, and the first intense tropical cycle of the season, Batsirai originated from a tropical disturbance that was first noted on 24 January. It fluctuated in intensity and became a moderate tropical storm on 27 January, after which it unexpectedly rapidly intensified into an intense tropical cycle. It then weakened and struggled to intensify through the coming days due to present wind hear and dry air, where it weakened after some time. Afterward, it entered much more favorable conditions, rapidly intensified yet again to a high-end Category 4 cycle on the Affair–Simpson scale while moving towards Madagascar. The storm underwent an eyeba